Based on the following links to create this notebook:

https://colab.research.google.com/drive/18LdlDDT87eb8cCTHZsXyS9ksQPzL3i6H

https://colab.research.google.com/drive/1tug_bpg8RwrFOI8C6Ed-zo0OgD3yfnWy#scrollTo=bhsj7BTPHepg

https://colab.research.google.com/drive/1tug_bpg8RwrFOI8C6Ed-zo0OgD3yfnWy


To run Gym, you have to install prerequisites like xvbf,opengl & other python-dev packages using the following codes.

https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html

In [ ]:
!pip install gym  torch
!apt-get install python-opengl ffmpeg -y 
!apt install xvfb -y 
!pip install pyvirtualdisplay  
!pip install piglet 
!pip install gym[box2d] 
!pip install tensorflow 


In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()


In [ ]:
# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) # error only
# import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay
from collections import namedtuple
from itertools import count

In [ ]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

def make_noise():
    mu, sigma = 0, 0.05 # mean and standard deviation
    return np.random.normal(mu, sigma, 1)[0]


# My Code

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle
import os

In [ ]:
class defaultActions:
    def __init__(self):

        self.main = [[0 , "Main off"],
                     [0.5 ,"Main slow"],
                     [0.75 , "Main meduim"],
                     [1    , "Main strong"]
                    ]
        self.left_right = [
                        [0, "Left Right off"],
                        [-0.5 , "Left slow"],
                        [-0.75  , "Left meduim"],
                        [-1 , "Left strong"],
                        
                        [0.5 , "Right slow"],
                        [0.75  , "Right meduim"],
                        [1 , "Right strong"],
                        ]
        self.all_actions = {}
        i = 0  
        for main_eng in self.main:
            for sec_eng in self.left_right:
#                 print (f"a{i},act:[{main_eng[0]},{sec_eng[0]},{main_eng[1]} {sec_eng[1]}]")
                self.all_actions[i] = [[main_eng[0],sec_eng[0]] , f"{main_eng[1]}, {sec_eng[1]}"]
                i+=1
        
    def get_full_action(self,id):
        return self.all_actions[id]

    def get_action(self,id,add_noise=True):
        if add_noise :
            return [self.all_actions[id][0][0]+make_noise(), self.all_actions[id][0][1]+make_noise()]
        else:
            return self.all_actions[id][0]

    def get_description(self,id):
        return self.all_actions[id][1]
    def get_action_count(self):
        return len(self.all_actions)
actions = defaultActions()
actions.get_full_action(21)


In [ ]:
class model(nn.Module):
    def __init__(self, layers, name=""):
        super(model, self).__init__()
        self.name = ""
        self.layers = layers
        self.features = nn.Sequential(*layers)


    def name(self):
        return self.name

    def model_summery(self):
        return self.features.summary()

    def forward(self, x):
        return self.features(x)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
input_size = 8
h1 = 500
h2 = 500
h3 = 500 
h4 = 500
h5 = 500
output_size = actions.get_action_count()
layers = [nn.Linear(input_size,h1) ,nn.ReLU(inplace=True), #, nn.BatchNorm1d(h1)
         nn.Linear(h1,h2) ,nn.ReLU(inplace=True),
#          nn.Linear(h2,h3) ,nn.ReLU(inplace=True),
#          nn.Linear(h3,h4) , nn.ReLU(inplace=True),
#          nn.Linear(h4,h5) ,nn.ReLU(inplace=True),
         nn.Linear(h5,output_size),nn.ReLU(inplace=True) # , nn.BatchNorm1d(output_size)
         ]


In [ ]:

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
# Box2d Environment
env = gym.make('LunarLanderContinuous-v2')
env.reset()
plt.imshow(env.render('rgb_array'))


In [ ]:
state_size = env.observation_space
print("state size is:", state_size)
a = env.action_space
print("action size=",a) 
state = env.reset()
print(state)   

# Action Space
            #is two floats [main engine, left-right engines].
            # Main engine: -1..0 off, 0..+1 throttle from 50% to 100% power. Engine can't work with less than 50% power.
            # Left-right:  -1.0..-0.5 fire left engine, +0.5..+1.0 fire right engine, -0.5..0.5 off
            self.action_space = spaces.Box(-1, +1, (2,), dtype=np.float32)

In [ ]:
df = 0.9 #discount_factor
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
max_iterations = 3000
steps_done = 0
TARGET_UPDATE = 10
BATCH_SIZE =20

# torch.autograd.set_detect_anomaly(True)
#mytestmodel.zero_grad()


In [ ]:
policy_net =  model(layers).to(device)
target_net =  model(layers).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.RMSprop(policy_net.parameters(), lr=5e-6, )
dtype = torch.float

In [ ]:

for a in range(100):
    memory = ReplayMemory(10000)
    print("Episode: ",a,"\n","-"*50)
    env = gym.make('LunarLanderContinuous-v2')
    env.reset()
    env = wrap_env(env)
    done = False
    iter = 0
#     print(done)
    observation = state = env.reset()

    action = 1
    TotalReward = 0

    while not done and iter < max_iterations :
      iter +=1
      
    #   action = env.action_space.sample()
      if len(memory) < BATCH_SIZE:

          sample = random.random()
          eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
          steps_done += 1
          state_values = policy_net.forward(torch.from_numpy(observation))
          if sample > eps_threshold:
                # t.max(1) will return largest column value of each row.
                # second column on max result is index of where max element was
                # found, so we pick action with the larger expected reward.
              real_action = state_values.argmax()

          else:
#               print("random sample")
              real_action = torch.tensor(random.randrange(actions.get_action_count()), device=device, dtype=torch.long)


          action = actions.get_action(real_action.item())
          observation, reward, done, _ = env.step(action)

          next_values = target_net.forward(torch.from_numpy(observation))
          if done :
                break
          observation, new_reward, done, _ = env.step(action)
    #       state_values[torch.isnan(state_values)] = 0
    #       next_values[torch.isnan(next_values)] = 0
          next_val = next_values.max() if next_values.max()!=0 and next_values.max() is not None else -0.1

          label = torch.tensor(reward + (df* new_reward), dtype=torch.float)

          print(iter,"action is:",actions.get_full_action(real_action.item()) ,
                 "reward: ",reward,"full:",label,
                "state_value:",state_values,
                "max opt",state_values.gather(-1,real_action))
#           print(real_action.unsqueeze(0))
          loss = F.smooth_l1_loss(state_values.gather(-1,real_action),label ) # Huber .unsqueeze(0)
    #       print (loss)
          optimizer.zero_grad()

          if loss != None:
              loss.backward()
          optimizer.step()


          TotalReward+= reward
      #print("state is:", observation)

      #if you want to see results on real-time 'open' the following 4 lines
#       if iter % TARGET_UPDATE == 0:
#           screen = env.render(mode='rgb_array')
#           plt.imshow(screen)
#           ipythondisplay.clear_output(wait=True)
#           ipythondisplay.display(plt.gcf())


      if iter % TARGET_UPDATE == 0:
            target_net.load_state_dict(policy_net.state_dict())
    print(TotalReward,iter,done)
    print("Episode: ",a,"\n","-"*50)


In [ ]:
env.close()
show_video()

In [ ]:
print(iter)

In [ ]:
#Draw random samples from a normal (Gaussian) distribution.
mu, sigma = 0, 0.05 # mean and standard deviation
s = np.random.normal(mu, sigma, 1)
print(s)